# Taller de Carga y Exploración Simple de de Datos


In [ ]:
try:
    import google.colab
    !pip uninstall matplotlib -y
    !pip install -q condacolab
    
    import condacolab
    condacolab.install_mambaforge()
except ModuleNotFoundError:
    pass

In [ ]:
try:
    import google.colab
    !git clone https://github.com/zorzalerrante/aves.git aves_git
    !mamba env update --name base --file aves_git/environment-colab.yml
except ModuleNotFoundError:
    pass

In [ ]:
!cd aves_git
!make download-external
!cd ..

In [ ]:
import sys
from pathlib import Path

AVES_ROOT = Path("..") if not 'google.colab' in sys.modules else Path("aves_git")
DATA_PATH = AVES_ROOT / "data" / "external"
DATA_PATH

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from gensim.utils import deaccent

sns.set(
    style="whitegrid", context="paper", font_scale=0.8, font="Fira Sans Extra Condensed"
)
mpl.rcParams["figure.dpi"] = 120


Para entender la CASEN:

http://observatorio.ministeriodesarrollosocial.gob.cl/storage/docs/casen/2017/Libro_de_Codigos_Casen_2017.pdf
http://observatorio.ministeriodesarrollosocial.gob.cl/storage/docs/casen/2020/Libro_de_codigos_Base_de_Datos_Casen_en_Pandemia_2020.pdf

Códigos de comunas:
http://www.subdere.gov.cl/documentacion/c%C3%B3digos-%C3%BAnicos-territoriales-actualizados-al-06-de-septiembre-2018

In [ ]:
casen = pd.read_stata(DATA_PATH / "casen_2017/Casen 2017.dta", convert_categoricals=False)
casen.head()


In [ ]:
comunas = casen["comuna"].unique().astype(int)
comunas


In [ ]:
len(comunas)


In [ ]:
casen_2020 = pd.read_stata(
    DATA_PATH / "casen_2020/Casen en Pandemia 2020 STATA.dta"
)
casen_2020.head()


In [ ]:
elecciones = pd.read_csv(
    DATA_PATH / "presidenciales_2021/Servel_20211121_PRESIDENCIALES_CHILE.csv",
    sep=";",
)
elecciones.head()


In [ ]:
codes = pd.read_excel(
    DATA_PATH / "CUT_2018_v04.xls"
)
codes


In [ ]:
elecciones.columns

In [ ]:
elecciones["votos_preliminar_string"].astype(int)


In [ ]:
elecciones["votos_preliminar_string"].str.strip().astype(int)


In [ ]:
for idx, row in elecciones.iterrows():
    try:
        int(row["votos_preliminar_string"])
    except:
        print(repr(row["votos_preliminar_string"]))
        break


In [ ]:
for idx, row in elecciones.iterrows():
    try:
        int(row["votos_preliminar_string"].strip())
    except:
        print(repr(row["votos_preliminar_string"].strip()))
        break


In [ ]:
elecciones["votos_preliminar_string_strip"] = (
    elecciones["votos_preliminar_string"].str.strip().str.replace(r"^$", "0").astype(int)
)


In [ ]:
elecciones["votos_preliminar_string_strip"].describe()


In [ ]:
elecciones.groupby("candidato")["votos_preliminar_string_strip"].sum()


In [ ]:
elecciones["comuna_nombre"]


In [ ]:
elecciones["comuna_nombre"] = elecciones["comuna_nombre"].str.strip()
elecciones["comuna_nombre"]

In [ ]:
turnout = (
    elecciones.groupby(["comuna_nombre", "candidato"])["votos_preliminar_string_strip"]
    .sum()
    .unstack()
)
turnout.head()


In [ ]:
turnout.columns

In [ ]:
turnout.columns = list(map(lambda x: x.strip(), turnout.columns))
turnout.columns


In [ ]:
turnout.sort_values("YASNA PROVOSTE CAMPILLAY", ascending=False)


In [ ]:
codes["comuna_nombre"] = codes["Nombre Comuna"].str.upper().map(deaccent)
codes["comuna_nombre"]


In [ ]:
casen_comunas = casen.join(
    codes.set_index("Código Comuna 2018")["comuna_nombre"], on="comuna"
)
casen_comunas


In [ ]:
casen_2020["comuna_nombre"] = casen_2020["comuna"].str.upper().map(deaccent)


In [ ]:
poblacion_comunas = (
    casen_2020[casen_2020["edad"] >= 17].groupby("comuna_nombre")["expr"].sum()
)
poblacion_comunas


In [ ]:
poblacion_comunas.sum()


In [ ]:
candidatos = [
    "EDUARDO ARTES BRICHETTI",
    "GABRIEL BORIC FONT",
    "JOSE ANTONIO KAST RIST",
    "MARCO ENRIQUEZ-OMINAMI GUMUCIO",
    "FRANCO PARISI FERNANDEZ",
    "YASNA PROVOSTE CAMPILLAY",
    "SEBASTIAN SICHEL RAMIREZ",
]



In [ ]:
turnout = (
    turnout.reset_index()
    .assign(comuna_nombre=lambda x: x["comuna_nombre"].map(deaccent))
    .set_index("comuna_nombre")[candidatos]
)


In [ ]:
turnout

In [ ]:
turnout.sum()


In [ ]:
comunas_x_ingreso = (
    casen_comunas.groupby(["comuna_nombre", "dautr"])["expr"]
    .sum()
    .unstack(fill_value=0)
)
comunas_x_ingreso


In [ ]:
comunas_index = turnout.join(comunas_x_ingreso, how="inner").index
comunas_index


In [ ]:
casen_comunas["grupo_etareo"] = pd.cut(
    casen_comunas["edad"], bins=[-1, 17, 25, 35, 45, 55, 65, 75, 85, 95, 120]
)
comunas_x_edad = (
    casen_comunas.groupby(["comuna_nombre", "grupo_etareo"])["expr"]
    .sum()
    .unstack(fill_value=0)
)
comunas_x_edad


In [ ]:
comunas_x_oficio = (
    casen_2020.groupby(["comuna_nombre", "oficio1_88"])["expr"]
    .sum()
    .unstack(fill_value=0)
)
comunas_x_oficio


In [ ]:
from aves.features.utils import normalize_rows

normalized_turnout = turnout.pipe(normalize_rows)
normalized_work = comunas_x_oficio.pipe(normalize_rows)

In [ ]:
import seaborn as sns


sns.clustermap(
    normalized_turnout.join(normalized_work)
    .corr()
    .loc[normalized_turnout.columns, normalized_work.columns], center=0, annot=True
)


In [ ]:
comunas_x_edad = (
    casen_2020[casen_2020["edad"] >= 17]
    .assign(
        grupo_etareo=lambda x: pd.cut(
            x["edad"], bins=[17, 25, 35, 45, 55, 65, 75, 85, 95, 120]
        )
    )
    .groupby(["comuna_nombre", "grupo_etareo"])["expr"]
    .sum()
    .unstack(fill_value=0)
)
comunas_x_edad


In [ ]:
normalized_edad = comunas_x_edad.pipe(normalize_rows)

In [ ]:
sns.clustermap(
    normalized_turnout.join(normalized_edad)
    .corr()
    .loc[normalized_turnout.columns, normalized_edad.columns], center=0, annot=True
)